Итоговая работа должна отвечать следующим требованиям:

	1. Программа реализована на языке программирования Python.
	2. Выполнена загрузка и чтение данных из файлов.
	3. Выполнена предварительная обработка данных (очистка и форматирование данных).
	4. Выполнен разведочный анализ данных EDA.
	5. Определены важные признаки. Выполнен поиск взаимосвязей между признаками.
	6. Выполнено уменьшение размерности массива данных.
	7. Составлена гипотеза о данных и выполнена проверка соответствующей гипотезы.

Аналитическая задача: построить модель для прогнозирования стоимости квартиры.

Исходные данные: имеется выборка данных о продаже квартир.
* *airports_nearest*		расстояние до ближайшего аэропорта в метрах (м)

* ***balcony***				число балконов
* ***ceiling_height***			высота потолков (м)
* ***cityCenters_nearest***		расстояние до центра города (м)
* ***floor***				этаж
* ***floors_total***			всего этажей в доме
* ***is_apartment***			апартаменты (булев тип)
* ***kitchen_area***			площадь кухни в квадратных метрах (м²)
* ***last_price***			цена на момент снятия с публикации
* ***living_area***			жилая площадь в квадратных метрах(м²)
* ***open_plan***			свободная планировка (булев тип)
* ***parks_around3000***		число парков в радиусе 3 км
* ***parks_nearest***			расстояние до ближайшего парка (м)
* ***ponds_around3000***		число водоёмов в радиусе 3 км
* ***ponds_nearest***			расстояние до ближайшего водоёма (м)
* ***rooms***				число комнат
* ***studio***				квартира-студия (булев тип)
* ***total_area***			площадь квартиры в квадратных метрах (м²)

**1. 	Программа реализована на языке программирования Python.**

In [ ]:
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

Загружаем необходимые библиотеки и модули

In [398]:
from pandas import *
import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from pandas_profiling import ProfileReport
from scipy import stats as st
from matplotlib.cbook import boxplot_stats

**2. 	Выполненяем загрузку и чтение данных из файлов.**

Загружаем Dataframe


In [399]:
df = read_csv("/content/drive/MyDrive/data/home_price.csv")

**3. Выполняем предварительную обработку данных (очистка и форматирование данных).**

In [400]:
df.head()

,last_price,total_area,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest
0,13000000.0,108.0,3,2.70,16.0,51.0,8,NaN,False,False,25.0,NaN,18863.0,16028.0,1.0,482.0,2.0,755.0
1,3350000.0,40.4,1,NaN,11.0,18.6,1,NaN,False,False,11.0,2.0,12817.0,18603.0,0.0,NaN,0.0,NaN
2,5196000.0,56.0,2,NaN,5.0,34.3,4,NaN,False,False,8.3,0.0,21741.0,13933.0,1.0,90.0,2.0,574.0
3,64900000.0,159.0,3,NaN,14.0,NaN,9,NaN,False,False,NaN,0.0,28098.0,6800.0,2.0,84.0,3.0,234.0
4,10000000.0,100.0,2,3.03,14.0,32.0,13,NaN,False,False,41.0,NaN,31856.0,8098.0,2.0,112.0,1.0,48.0


Первый взгляд на DataFraame

In [ ]:
df.info()

In [ ]:
df.describe(include = "all")

Присутствуют аномальные значения: last_price(min), last_price(max), rooms(min), ceiling_height(min),ceiling_height(max), living_area(min), kitchen_area(min)

Проверим DataFrame на дубликаты

In [ ]:
df.duplicated().sum()

удалим дубликаты

In [404]:
df=df.drop_duplicates()

проверка

In [ ]:
df.duplicated().sum()

дубликатов нет

также мы видим много пропусков данных в строках. Посмотрим на них внимательнее

In [ ]:
df.isnull().describe()

Посмотрим - сколько пропущенных значений в каждой колонке.

In [ ]:
df.isnull().sum()

Преобразуем отсутствующие данные в столбцах в нулевые значения

In [408]:
df['is_apartment'] = df['is_apartment'].fillna(value=0).astype(int)
df['balcony'] = df['balcony'].fillna(value=0).astype(int)
df['parks_around3000'] = df['parks_around3000'].fillna(value=0).astype(int)
df['ponds_around3000'] = df['ponds_around3000'].fillna(value=0).astype(int)

Заменим во всех столбцах False на 0, а True на 1

In [409]:
df = df.replace({True: 1, False: 0})

In [ ]:
df.head()

Удалим строки где нет информации об этажности домов

In [411]:
df.dropna(subset=['floors_total'], inplace=True)

Заменим пустые значения жилой площади и кухни, для этого получим коэффициент отношения среднего значения жилой площади к среднему значению общей площади и умножим на общую площадь объекта.


In [412]:
df['living_area'].fillna(df['total_area']*(df['living_area'].mean() / df['total_area'].mean()), inplace=True)
df['kitchen_area'].fillna(df['total_area']*(df['kitchen_area'].mean() / df['total_area'].mean()), inplace=True)

Заменим пустые значения высоты потолков на медианное значение

In [413]:
df['ceiling_height'].fillna(df['ceiling_height'].median(), inplace=True)

В связи с тем что удаленность от прудов и парков имеет больше половины пропусков, данные столбцы удаляем из DataFrame

In [414]:
df.drop(['parks_nearest', 'ponds_nearest'], axis= 1 , inplace= True )

Так же удалим столбец удаленности от аэропорта

In [415]:
df.drop(['airports_nearest'], axis= 1 , inplace= True )

Удалим отсутствующие записи удаленности от центра, так как некорректные данные смогут внести корректировку в определение стоимости квартиры

In [416]:
df.dropna(subset=['cityCenters_nearest'], inplace=True)

Добавим столбец стоимость кв.м.

In [417]:
df['стоимость кв.м.'] = df['last_price']/df['total_area']
df['стоимость кв.м.']=df['стоимость кв.м.'].astype('int32')

Добавим столбец c типом этажа



In [418]:
def floor_lv(row):
    if row['floor'] == 1:
        return 'первый'
    elif row['floor'] == row['floors_total']:
        return 'последний'
    else:
        return 'средний'
df['floor_level'] = df.apply(floor_lv, axis=1)


Потолки

In [419]:
df['ceiling_height'].describe()

count    18104.000000
mean         2.737167
std          0.988372
min          1.000000
25%          2.600000
50%          2.650000
75%          2.730000
max        100.000000
Name: ceiling_height, dtype: float64

Посмотрим на аномальные данные

In [ ]:
df[df['ceiling_height']>4]

In [ ]:
df[df['ceiling_height']<2.5]

минимальные значения высоты потока прописаны в СП 54.13330.2016 «СНиП 31-01-2003 Здания жилые многоквартирные». В своде правил установлено, что в многоквартирных домах в центральной части России потолок должен быть не ниже 2,5 м, а в Сибири, на Дальнем Востоке и на севере Центральной России — не ниже 2,7 м. Максимальная высота потолков в элитной застройке  до 4 метров в зависимости от площади квартиры. В связи с тем что квартиры в DataFrame вносились в ручную, возможно была допущена ошибка. Внесем изменение и установим для этих квартир высоту потолков 2.6м.(как среднюю между 2.5 и 2.7)

In [422]:
df.loc[(df['ceiling_height'] < 2.5), 'ceiling_height'] = 2.6
df.loc[(df['ceiling_height'] > 4), 'ceiling_height'] = 2.6

Создадим столбец удаленности от центра в км.

In [423]:
df['cityCenters_nearest, км.']=df['cityCenters_nearest']/1000

In [424]:
df['cityCenters_nearest, км.'] = df['cityCenters_nearest, км.'].round(1)
df = df.drop('cityCenters_nearest', axis=1)

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

**Стоимость квартир**

In [427]:
df['last_price'].describe()

count    1.810400e+04
mean     7.574629e+06
std      1.215979e+07
min      1.219000e+04
25%      4.000000e+06
50%      5.250000e+06
75%      7.690000e+06
max      7.630000e+08
Name: last_price, dtype: float64

Квартира стоимостью 12 тыс.

In [ ]:
df[df['last_price']<100000]

На лицо ошибка введения данных. Посмотрим аналогичные квартиры на удаленности 9 км от центра

In [ ]:
df.loc[(df['cityCenters_nearest, км.'] > 9) & (df['cityCenters_nearest, км.'] <= 10)& (df['total_area'] > 108)& (df['total_area'] <= 110)]

видно что при внесении данных пропустили 000. Внесем изменение в 2- последних столбца.

In [430]:
df.loc[(df['last_price'] == 12190.0), 'last_price'] = 12190000.0

In [ ]:
px.box(df, x='last_price')

Удалим выбросы, в данном случае и в остальных будем удалять выбросы основываясь на квартили с учетом 1.5 размаха

In [432]:
outliers = boxplot_stats(df["last_price"]).pop(0)['fliers']
df = df[~df["last_price"].isin(outliers)]

**Общая площадь**

In [433]:
df['total_area'].describe()

count    16530.000000
mean        55.649750
std         21.119883
min         12.000000
25%         40.000000
50%         51.000000
75%         66.800000
max        255.000000
Name: total_area, dtype: float64

In [ ]:
px.box(df, x='total_area')

Удалим выбросы

In [435]:
outliers = boxplot_stats(df["total_area"]).pop(0)['fliers']
df = df[~df["total_area"].isin(outliers)]

**Жилая площадь**

In [ ]:
df['living_area'].describe()

Согласно действующим СНиПам, минимальная площадь комнаты должна быть не менее 8 квадратных метров (п. 5.3 СП 5.413330.2011), при этом её ширина не должна бы ть меньше 2,25 метра (п 6.1.9 СП 31-107-2004).

In [437]:
df.loc[(df['living_area'] < 8), 'living_area'] = 8

In [ ]:
px.box(df, x='living_area')

Удалим выбросы

In [439]:
outliers = boxplot_stats(df["living_area"]).pop(0)['fliers']
df = df[~df["living_area"].isin(outliers)]

**Кухни**

In [ ]:
df['kitchen_area'].describe()

по СНиПам ширина кухни должна быть не менее 1,8 м, более удобная 2,4 м. Минимальная площадь кухни 8 кв.м, рекомендована 12 кв.м.

In [441]:
df.loc[(df['kitchen_area'] < 8), 'kitchen_area'] = 8

In [ ]:
px.box(df, x='kitchen_area')

Удалим выбросы

In [443]:
outliers = boxplot_stats(df["kitchen_area"]).pop(0)['fliers']
df = df[~df["kitchen_area"].isin(outliers)]

**Комнаты**

In [444]:
df['rooms'].describe()

count    15106.000000
mean         1.923209
std          0.885685
min          0.000000
25%          1.000000
50%          2.000000
75%          3.000000
max          6.000000
Name: rooms, dtype: float64

In [ ]:
px.box(df, x='rooms')

Расставим столбцы в нужном (для меня) порядке

In [446]:
df = df[['floor', 'floor_level', 'floors_total', 'rooms', 'living_area', 'kitchen_area', 'total_area', 'ceiling_height',
'balcony', 'is_apartment', 'studio', 'open_plan', 'cityCenters_nearest, км.',
'parks_around3000', 'ponds_around3000', 'last_price', 'стоимость кв.м.']]

In [447]:
df.head()

,floor,floor_level,floors_total,rooms,living_area,kitchen_area,total_area,ceiling_height,balcony,is_apartment,studio,open_plan,"cityCenters_nearest, км.",parks_around3000,ponds_around3000,last_price,стоимость кв.м.
1,1,первый,11.0,1,18.6,11.0,40.4,2.65,2,0,0,0,18.6,0,0,3350000.0,82920
2,4,средний,5.0,2,34.3,8.3,56.0,2.65,0,0,0,0,13.9,1,2,5196000.0,92785
6,6,средний,26.0,1,10.6,14.4,37.3,2.65,1,0,0,0,19.1,0,0,3700000.0,99195
9,7,средний,9.0,3,43.6,8.0,61.0,2.50,2,0,0,0,15.0,0,0,5400000.0,88524
10,3,средний,12.0,1,20.3,8.5,39.6,2.67,0,0,0,0,13.9,1,2,5050000.0,127525


Дл удобства работы выведем все столбцы на печать

In [ ]:
df.keys()

In [ ]:
df.info()

**4. Проведем разведочный анализ данных EDA. Воспользуемся библиотекой pandas_profiling**

In [ ]:
profile = ProfileReport(df)
profile.to_notebook_iframe()

Подведем первые итоги:
1. Квартиры расположены с 1 по 33 этаж,
2. Дома в которых размещены эти квартиры от одно до 60-ти этажного.
3. Количество комнат от 0 до 6.
4. Жилая площадь от 8 до 89.1 кв.м.
5. Площадь кухни от 8 до 48.3 кв.м.
6. Общая площадь от 12 до 107 кв.м.
7. Высота потолков от 2.5 до 4м.
8. Балконы от 0 до 5шт.
9. Аппартаменты 31шт.
10. Студий 90шт.
11. Открытой планировки  - 35шт.
12. Расстояние до центра от 0.2 до 66км. Возможно данная подборка объявлений со всего района или области.
13. Парки на расстоянии 3км.: от 0 до 3
14. Пруды на расстоянии 3км.: от 0 до 3
15. Стоимость квартиры от 1190 000 до	13 200 000.
16. Стоимость квадратного метра от	17 391 до 387 096.

5. Определяем важные признаки. Ищем взаимосвязей между признаками.

Количество предложений квартир по этажам

In [ ]:
df['floor'].value_counts()

In [ ]:
df['floor'].value_counts(normalize=True).plot.pie()

In [ ]:
px.histogram(df, x='floor')

In [454]:
print(round((df[df['floor']<=10]['last_price'].count()/len(df))*100, 1), '% квартир с 1 по 10 этаж')

84.1 % квартир с 1 по 10 этаж


In [ ]:
px.scatter(df, x='last_price', y='floor')

На графике видно что стоимость квартир на верхних этажах ниже, необходимо посмотреть данный график с учетом площади.

In [ ]:
px.scatter(df, x='last_price', y='floor', color='total_area')

Посмотрим как зависит стоимость квартиры от этажа

In [ ]:
sb.lineplot(df, x='floor_level', y='last_price')

Распределение предложений по количеству этажей в доме

In [458]:
df['floors_total'].value_counts()

5.0     3207
9.0     3050
12.0    1047
16.0    1041
10.0     793
25.0     779
4.0      643
17.0     595
6.0      468
14.0     423
24.0     342
7.0      315
3.0      288
18.0     278
23.0     261
19.0     234
15.0     219
8.0      168
22.0     153
20.0     153
13.0     151
11.0     112
26.0     101
21.0     100
27.0      83
2.0       60
28.0      15
35.0      15
1.0        5
36.0       2
34.0       1
60.0       1
29.0       1
52.0       1
37.0       1
Name: floors_total, dtype: int64

In [ ]:
df['floors_total'].value_counts(normalize=True).plot.pie()

40% - это 5 и 9 этажные дома

In [ ]:
px.histogram(df, x='floors_total', color='floor')

In [ ]:
sb.scatterplot(df, x='floors_total', y='floor')

In [ ]:
px.scatter(df, x='last_price', y='floor', color='floors_total')


На графике заметно что большинство квартиры в расположены до 15 этажа.

In [463]:
round(df[df['floors_total']<=15]['last_price'].value_counts().sum()/len(df)*100, 2) # Процент  домов до 10 этажа

72.48

In [464]:
round(df.loc[(df['floors_total'] > 15) & (df['floors_total'] <= 30)]['last_price'].value_counts().sum()/len(df)*100, 2) # Процент  домов от 10 до 20 этажа

27.38

In [465]:
round(df.loc[(df['floors_total'] > 30) & (df['floors_total'] <= 60)]['last_price'].value_counts().sum()/len(df)*100, 2) # Процент  домов от 30 до 60 этажа

0.14

Посмотрим что у нас с количеством комнат

In [ ]:
df['rooms'].value_counts()

In [ ]:
df['rooms'].value_counts(normalize=True).plot.pie()

1-2-3 комнатные квартиры занимают 92% всех объявлений. 1 и 2-х комнатные по ~34%, 3-х комнатные - ~25%, на долю 4-х комнатных квартир приходиться ~6%.

In [ ]:
px.box(df, x="rooms")

In [ ]:
px.histogram(df, x='rooms')

In [ ]:
px.scatter(df, x='last_price', y='rooms')

Стоимость квартир растет при увеличении количества комнат, но необходимо учитывать что в данном графике не учтены факторы близости к центру и площади квартир

Жилая площадь

In [471]:
df['living_area'].value_counts()

18.000000    622
17.000000    418
20.000000    382
30.000000    368
16.000000    329
            ... 
17.430000      1
16.040000      1
42.910000      1
27.871672      1
12.600000      1
Name: living_area, Length: 1342, dtype: int64

In [ ]:
df['living_area'].plot.density()

In [ ]:
px.histogram(df, x='living_area', color='rooms')

На графике видно явное распределение жилой площади к количеству комнат, основная часть данных распологается до 60 кв.м. График интерактивный, можно отключать количество комнат через легенду.

In [ ]:
px.scatter(df, x='last_price', y='living_area', color='rooms', trendline="ols")

При увеличении жилой площади увеличивается стоимость квартиры.

В DataFrame присутствуют квартиры с количеством комнат = 0, посмотрим на них.

In [475]:
df[df['rooms']==0]['studio'].value_counts()


1    85
0    30
Name: studio, dtype: int64

In [476]:
df['studio'].value_counts(normalize=True)*100

0    99.40421
1     0.59579
Name: studio, dtype: float64

In [477]:
df[df['rooms']==0]['open_plan'].value_counts()

0    85
1    30
Name: open_plan, dtype: int64

In [478]:
df['open_plan'].value_counts(normalize=True)*100

0    99.774924
1     0.225076
Name: open_plan, dtype: float64

это квартиры студии и открытой планировки. Всего таких квартир studio - 0,56% и open_plan - 0,22%

Посмотрим на кухни

In [479]:
df['kitchen_area'].value_counts()

8.000000     6470
10.000000     921
9.000000      633
11.000000     573
12.000000     472
             ... 
9.620000        1
10.920000       1
10.580000       1
13.428063       1
14.058324       1
Name: kitchen_area, Length: 523, dtype: int64

In [ ]:
px.histogram(df, x='kitchen_area', color='rooms')

In [ ]:
px.scatter(df, x='last_price', y='kitchen_area', trendline="ols")

с увеличением кухни стоимость квартиры увеличивается

Посмотрим на общую площадь квартир

In [482]:
df['total_area'].value_counts()

45.00     331
42.00     279
31.00     261
60.00     255
44.00     240
         ... 
59.06       1
102.80      1
90.37       1
96.60       1
66.86       1
Name: total_area, Length: 1198, dtype: int64

In [ ]:
px.histogram(df, x='total_area', color='rooms')

Большинство квартир до 90 кв.м.


In [ ]:
px.scatter(df, x='last_price', y='total_area', color='rooms', trendline="ols")

На данном графике видно что при увеличении площади квартиры увеличивается и стоимость

Высота потолков

In [485]:
df['ceiling_height'].value_counts()

2.65    6060
2.50    2481
2.60    1227
2.70    1026
2.55     737
        ... 
3.09       1
3.37       1
3.98       1
3.57       1
2.91       1
Name: ceiling_height, Length: 116, dtype: int64

In [486]:
df['ceiling_height'].describe()

count    15106.000000
mean         2.674991
std          0.179428
min          2.500000
25%          2.600000
50%          2.650000
75%          2.700000
max          4.000000
Name: ceiling_height, dtype: float64

In [ ]:
px.histogram(df, x='ceiling_height')

Наличие потолков выше 3.5 метров вызывает подозрение особенно max 100м, так же как и min 1м. Посмотрим на мин высоту квартир

In [ ]:
df['ceiling_height'].plot.density()

In [ ]:
px.scatter(df, x='last_price', y='ceiling_height', color='rooms', trendline="ols")

Стоимость квартир незначительно зависит от высота потолка

Балконы

In [490]:
df['balcony'].value_counts()

0    9708
1    2704
2    2394
5     176
4      86
3      38
Name: balcony, dtype: int64

In [ ]:
px.histogram(df, x='balcony')

In [ ]:
px.scatter(df, x='last_price', y='balcony', trendline="ols")

Наличие балконов незначительно влияет на стоимость квартир, квартиры с балконом (балконами) стоит дороже.

Апартаменты

In [ ]:
df['is_apartment'].value_counts()

In [ ]:
df['is_apartment'].value_counts(normalize=True)

In [ ]:
df['is_apartment'].value_counts(normalize=True).plot.pie()

Апартаменты в данном DataFrame занимают всего 0.2%

In [ ]:
px.scatter(df, x='last_price', y='is_apartment', color='total_area')

Стоимость апартаментов ниже, чем обычной квартиры, и предложений тоже очень мало

Студии

In [ ]:
df['studio'].value_counts()

In [ ]:
df['studio'].value_counts(normalize=True)

In [ ]:
df['studio'].value_counts(normalize=True).plot.pie()

Студии в данном DataFrame занимают всего 0.6%

In [ ]:
px.scatter(df, x='last_price', y='studio')

Стоимость студии ниже, чем обычной квартиры, и предложений тоже очень мало

Открытая планировка

In [ ]:
df['open_plan'].value_counts()

In [ ]:
df['open_plan'].value_counts(normalize=True)

In [ ]:
df['open_plan'].value_counts(normalize=True).plot.pie()

Квартиры с открытой планировкой в данном DataFrame занимают всего 0.2%

In [ ]:
px.scatter(df, x='last_price', y='open_plan', color='total_area')

Стоимость квартиры с открытой планировкой ниже, чем обычной квартиры, и предложений тоже очень мало

Расстояние до центра, км


In [ ]:
df['cityCenters_nearest, км.'].value_counts(0)

In [ ]:
px.histogram(df, x='cityCenters_nearest, км.', color='rooms')

Всплески на графике явно указывают на населенные пункты

In [ ]:
df.keys()

посмотрим на соотношение стоимости и близости к центру

In [ ]:
px.scatter(df, x='last_price', y='cityCenters_nearest, км.', color='rooms', trendline="ols")

In [ ]:
px.scatter(df, x='last_price', y='cityCenters_nearest, км.', color='total_area', trendline="ols")

Чем дальше от центра тем стоимость равнозначной квартиры ниже

Наличие рядом парков

In [ ]:
df['parks_around3000'].value_counts()

In [ ]:
df['parks_around3000'].value_counts(normalize=True)

In [ ]:
df['parks_around3000'].value_counts(normalize=True).plot.pie()

In [ ]:
px.histogram(df, x='last_price', color='parks_around3000')

In [ ]:
px.scatter(df, x='last_price', y='parks_around3000', trendline="ols")

На графике видно что наличие парков увеличивает стоимость квартир

In [ ]:
px.scatter(df, x='cityCenters_nearest, км.', y='parks_around3000', trendline="ols")

Чем дальше от центра тем меньше парков

Наличие рядом прудов

In [ ]:
df['ponds_around3000'].value_counts()

In [ ]:
df['ponds_around3000'].value_counts(normalize=True)

In [ ]:
df['ponds_around3000'].value_counts(normalize=True).plot.pie()

In [ ]:
px.histogram(df, x='last_price', color='ponds_around3000')

In [ ]:
px.scatter(df, x='last_price', y='ponds_around3000', trendline="ols")

Наличие прудов увеличивает стоимость квартир

In [ ]:
px.scatter(df, x='cityCenters_nearest, км.', y='ponds_around3000', trendline="ols")

Чем дальше от центра тем меньше количество прудов

Стоимость квартир

In [ ]:
df['last_price'].describe()

In [ ]:
px.histogram(df, x='last_price')

Основная часть квартир находится в ценовом диапазоне до 12млн.

In [ ]:
px.box(df, x='last_price')

In [ ]:
df['last_price'].plot.density()

In [ ]:
px.histogram(df, x='last_price', color='rooms')

In [ ]:
px.histogram(df, x='last_price', color='floor')

Стоимость кв.м.

In [ ]:
df['стоимость кв.м.'].describe()

In [ ]:
df['стоимость кв.м.'].plot.density()

In [ ]:
df.keys()

In [ ]:
px.scatter(df, x='стоимость кв.м.', y='floor', color='rooms')

In [ ]:
px.scatter(df, x='стоимость кв.м.', y='floors_total', color='rooms')

In [ ]:
px.scatter(df, x='стоимость кв.м.', y='floors_total', color='total_area')

In [ ]:
px.scatter(df, x='стоимость кв.м.', y='living_area', color='rooms')

С увеличением жилой площади увеличивается количество комнат

In [ ]:
px.scatter(df, x='стоимость кв.м.', y='total_area', color='rooms')

In [ ]:
px.scatter(df, x='стоимость кв.м.', y='ceiling_height', color='rooms')

In [ ]:
px.scatter(df, x='стоимость кв.м.', y='cityCenters_nearest, км.', color='rooms')

In [ ]:
px.scatter(df, x='last_price', y='total_area', color='floor', trendline="ols")

In [ ]:
px.scatter(df, x='cityCenters_nearest, км.', y='total_area', color='rooms', trendline="ols")

При приближении к центру увеличивается общая площадь квартир, соответственно увеличивается количество комнат в квартирах, и растет стоимость кв.м.

In [ ]:
px.scatter(df, y='стоимость кв.м.', x='last_price', trendline="ols")

In [ ]:
px.scatter(df, x='стоимость кв.м.', y='parks_around3000', trendline="ols")

In [ ]:
px.scatter(df, x='стоимость кв.м.', y='ponds_around3000', trendline="ols")

Наличие рядом парка или пруда незначительно увеличивает стоимость стоимость кв.м.

Вывод:
1. Цена зависит от площади квартиры (жилой, общей, кухни)и удаленности от центра.
2. Стоимость предложенных квартир расположеных в "середине" (не на крайних этажах) выше.
3. Незначительное влияние на стоимость квартир оказывает наличие балконов, парков и прудов.

6. Выполняем уменьшение размерности массива данных.

Уменьшим размер DataFrame

In [ ]:
df.info()

In [544]:
df['floor']=df['floor'].astype('int8')
df['floors_total']=df['floors_total'].astype('int8')
df['rooms']=df['rooms'].astype('int8')
df['living_area']=df['living_area'].astype('float16')
df['kitchen_area']=df['kitchen_area'].astype('float16')
df['total_area']=df['total_area'].astype('float16')
df['ceiling_height']=df['ceiling_height'].astype('float16')
df['balcony']=df['balcony'].astype('int8')
df['is_apartment']=df['is_apartment'].astype('int8')
df['studio']=df['studio'].astype('int8')
df['open_plan']=df['open_plan'].astype('int8')
df['balcony']=df['balcony'].astype('int8')
df['cityCenters_nearest, км.']=df['cityCenters_nearest, км.'].astype('float16')
df['balcony']=df['balcony'].astype('int8')
df['parks_around3000']=df['parks_around3000'].astype('int8')
df['ponds_around3000']=df['ponds_around3000'].astype('int8')
df['last_price']=df['last_price'].astype('int32')


In [ ]:
df.info()

**ВЫВОД: Задание по уменьшению DataFrame выполнено**

7. Составим гипотезу о данных и выполнена проверка соответствующей гипотезы.

выдвенем нулевую гипотезу, что стоимость квадратного метра квартиры не зависит наличия балкона

In [546]:
A = df['balcony']
B = df['стоимость кв.м.']
t_stat, p_value = st.ttest_ind(A,B)
print("Т-статистика:", t_stat)
print("р-значения:", p_value)
if p_value > 0.05:
  print('мы не отвергаем нулевую гипотезу что стоимость квадратного метра квартиры не зависит наличия балкона')
else:
  print('мы отвергаем нулевую гипотезу что стоимость квадратного метра квартиры не зависит наличия балкона')


Т-статистика: -526.522580094983
р-значения: 0.0
мы отвергаем нулевую гипотезу что стоимость квадратного метра квартиры не зависит наличия балкона


In [ ]:
df.describe()

In [ ]:
df['living_area'].describe()

In [549]:
df['last_price']=df['last_price']/1000000

In [550]:
df['last_price'] = df['last_price'].round(2)

In [551]:
df['ceiling_height'] = df['ceiling_height'].round(1)

In [ ]:
sb.lineplot(df, y='last_price', x='floor')

In [ ]:
sb.lineplot(df, y='last_price', x='floors_total')

In [ ]:
sb.lineplot(df, y='last_price', x='rooms')

создание вспомогательных столбцов

In [ ]:
df['total_area_d'] = Series()
df.loc[df['total_area'] <= 10, 'total_area_d'] = 10
df.loc[df['total_area'] > 100, 'total_area_d'] = 110
df.loc[(df['total_area'] > 10) & (df['total_area'] <= 20), 'total_area_d'] = 20
df.loc[(df['total_area'] > 20) & (df['total_area'] <= 30), 'total_area_d'] = 30
df.loc[(df['total_area'] > 30) & (df['total_area'] <= 40), 'total_area_d'] = 40
df.loc[(df['total_area'] > 40) & (df['total_area'] <= 50), 'total_area_d'] = 50
df.loc[(df['total_area'] > 50) & (df['total_area'] <= 60), 'total_area_d'] = 60
df.loc[(df['total_area'] > 60) & (df['total_area'] <= 70), 'total_area_d'] = 70
df.loc[(df['total_area'] > 70) & (df['total_area'] <= 80), 'total_area_d'] = 80
df.loc[(df['total_area'] > 80) & (df['total_area'] <= 90), 'total_area_d'] = 90
df.loc[(df['total_area'] > 90) & (df['total_area'] <= 100), 'total_area_d'] = 100
df['total_area_d'] =df['total_area_d'].astype('int16')

In [ ]:
df['cityCenters_z'] = Series()
df.loc[df['cityCenters_nearest, км.'] <= 5, 'cityCenters_z'] = 0
df.loc[(df['cityCenters_nearest, км.'] > 5) & (df['cityCenters_nearest, км.'] <= 10), 'cityCenters_z'] = 1
df.loc[(df['cityCenters_nearest, км.'] > 10) & (df['cityCenters_nearest, км.'] <= 15), 'cityCenters_z'] = 2
df.loc[(df['cityCenters_nearest, км.'] > 15) & (df['cityCenters_nearest, км.'] <= 20), 'cityCenters_z'] = 3
df.loc[(df['cityCenters_nearest, км.'] > 20) & (df['cityCenters_nearest, км.'] <= 25), 'cityCenters_z'] = 4
df.loc[(df['cityCenters_nearest, км.'] > 25) & (df['cityCenters_nearest, км.'] <= 30), 'cityCenters_z'] = 5
df.loc[(df['cityCenters_nearest, км.'] > 30) & (df['cityCenters_nearest, км.'] <= 35), 'cityCenters_z'] = 6
df.loc[(df['cityCenters_nearest, км.'] > 35) & (df['cityCenters_nearest, км.'] <= 40), 'cityCenters_z'] = 7
df.loc[(df['cityCenters_nearest, км.'] > 40) & (df['cityCenters_nearest, км.'] <= 45), 'cityCenters_z'] = 8
df.loc[(df['cityCenters_nearest, км.'] > 45) & (df['cityCenters_nearest, км.'] <= 50), 'cityCenters_z'] = 9
df.loc[(df['cityCenters_nearest, км.'] > 50) & (df['cityCenters_nearest, км.'] <= 55), 'cityCenters_z'] = 10
df.loc[(df['cityCenters_nearest, км.'] > 55) & (df['cityCenters_nearest, км.'] <= 60), 'cityCenters_z'] = 11
df.loc[df['cityCenters_nearest, км.'] > 60, 'cityCenters_z'] = 12
df['cityCenters_z'] =df['cityCenters_z'].astype('int16')

In [ ]:
sb.lineplot(df, y='last_price', x='total_area_d')

In [ ]:
sb.lineplot(df, y='last_price', x='balcony')

In [ ]:
sb.lineplot(df, y='last_price', x='parks_around3000')

In [ ]:
sb.lineplot(df, y='last_price', x='ponds_around3000')

In [ ]:
sb.lineplot(df, y='last_price', x='cityCenters_z')

**Создание модели для прогнозирования стоимости квартиры**

Введите пожелания по будующей квартире и мы посчитаем слолько она будет стоить (примерно)

In [571]:
rooms_k=int(input ("Введите количество комнат (от 0 до 6): "))
total_area_k=int(input ("Введите общую площадь: "))
balcony_k=int(input ("Введите количество балконов(от 0 до 5): "))
parks_around3000_k=int(input ("Введите количество парков (от 0 до 3): "))
ponds_around3000_k=int(input ("Введите количество прудов (от 0 до 3): "))
cityCenters_k=int(input ("Введите удаленность от центра: "))

# Определение коэффициентов
def rooms_kf(x, df_i):
    i=df[df['rooms']==x]['last_price'].mean()/df[df['rooms']==0]['last_price'].mean()
    return i

k_r=round(rooms_kf(rooms_k, df), 2)
#print('k_r=',k_r)

def area_kf(x, df_i):
    if x<=20: k=20
    elif (x>20 and x<=30): k=30
    elif (x>30 and x<=40): k=40
    elif (x>40 and x<=50): k=50
    elif (x>50 and x<=60): k=60
    elif (x>60 and x<=70): k=70
    elif (x>70 and x<=80): k=80
    elif (x>80 and x<=90): k=90
    elif (x>90 and x<=100): k=100
    else: k=4.5

    i=df[df['total_area_d']==k]['last_price'].mean()/df[df['total_area_d']==20]['last_price'].mean()
    return i

k_a=round(area_kf(total_area_k, df), 2)
#print('k_a=',k_a)

def balcony_kf(x, df_i):
    i=df[df['balcony']==x]['last_price'].mean()/df[df['balcony']==0]['last_price'].mean()
    return i

k_b=round(balcony_kf(balcony_k, df), 2)
#print('k_b=',k_b)

def park_kf(x, df_i):
    i=df[df['parks_around3000']==x]['last_price'].mean()/df[df['parks_around3000']==0]['last_price'].mean()
    return i

k_pr=round(park_kf(parks_around3000_k, df), 2)
#print('k_pr=', k_pr)

def ponds_kf(x, df_i):
    i=df[df['ponds_around3000']==x]['last_price'].mean()/df[df['ponds_around3000']==0]['last_price'].mean()
    return i

k_pd=round(ponds_kf(ponds_around3000_k, df), 2)
#print('k_pd=',k_pd)

def centr_kf(x, df_i):
    if x<=5: k=0
    elif (x>5 and x<=10): k=1
    elif (x>10 and x<=15): k=2
    elif (x>15 and x<=20): k=3
    elif (x>20 and x<=25): k=4
    elif (x>25 and x<=30): k=5
    elif (x>30 and x<=35): k=6
    elif (x>35 and x<=40): k=7
    elif (x>40 and x<=45): k=8
    elif (x>45 and x<=50): k=9
    elif (x>50 and x<=55): k=10
    elif (x>55 and x<=60): k=11
    else: k=12
    i=df[df['cityCenters_z']==k]['last_price'].mean()/df[df['cityCenters_z']==12]['last_price'].mean()
    return i

k_d=round(centr_kf(cityCenters_k, df), 2)
#print('k_d=',k_d)

st_kv=1.19*k_r*k_a*k_b*k_pr*k_pd*k_d
print('Примерная стоимость квартиры будет ', st_kv, 'млн.')

Введите количество комнат (от 0 до 6): 4
Введите общую площадь: 100
Введите количество балконов(от 0 до 5): 2
Введите количество парков (от 0 до 3): 2
Введите количество прудов (от 0 до 3): 1
Введите удаленность от центра: 6
Примерная стоимость квартиры будет  22.572383730624004 млн.
